In [1]:
import pandas as pd
import numpy as np
import pulp
import time
import sys

In [2]:
rng = np.random.RandomState(10)
n = int(sys.argv[0])
m = int(sys.argv[1])
d = rng.randint(1, 9, m)
t = rng.randint(24, 241, n)
c = rng.randint(50, 2001, m)
b = rng.randint(2000, 10001, n)
e = rng.randint(1, int(n + 1), m)
w = rng.randint(-10, 11, (m, n))

In [3]:
def alg1() :
    library = np.array(e)
    z = 0
    for k in range(n):
        index_1 = ['i_' + str(i) for i in range(m)]
        columns_1 = ['k_' + str(k)]
        matrix_1 = np.array(w)[:, k]
        table_1 = pd.DataFrame(matrix_1, index = index_1, columns = columns_1)
        table_1['avg'] = [table_1.iloc[i,0]/c[i]/d[i] for i in range(m)]
        table_1 = table_1.sort_values(by=['avg'], ascending=False)
        budget = 0
        duration = 0
        solution = []
        sum_solution = 0
        for i in range(table_1.shape[0]):
            weight = table_1.iloc[i, 0]
            item_no = int(table_1.index[i][2:])
            inventory = library[item_no]
            cost = c[item_no]
            time = d[item_no]
            if inventory > 0 and budget + cost <= b[k] and duration + time <= t[k] and sum_solution + weight >= sum_solution:
                budget += cost
                duration += time
                solution.append(item_no)
                sum_solution += weight
        z += sum_solution
    return z

In [4]:
def alg2():
    z = 0
    index_2 = ['k_' + str(k) for k in range(n)]
    columns_2 = ['i_' + str(i) for i in range(m)]
    matrix_2 = np.array(w).transpose()
    table_2 = pd.DataFrame(matrix_2, index = index_2, columns = columns_2)
    table_2['avg'] = table_2.mean(axis=1)
    table_2 = table_2.sort_values(by=['avg'], ascending=False)
    budget = {}
    duration = {}
    solution ={}
    sum_solution={}
    for k in range(n):
        budget["k_{0}".format(k)] = 0
        duration["k_{0}".format(k)] = 0
        solution["k_{0}".format(k)] = []
        sum_solution["k_{0}".format(k)] = 0
    for i in range(m):
        item_no = int(table_2.columns[i][2:])
        inventory = np.array(e)[item_no]
        cost = c[item_no]
        time = d[item_no]
        table_2 = table_2.sort_values(by=['i_'+str(item_no)], ascending=False)
        k_loop = 0
        while inventory != 0 and k_loop < n:
            weight = table_2.iloc[k_loop,i]
            knap = int(table_2.index[k_loop][2:])
            the_k = 'k_' + str(knap)
            if inventory > 0 and budget[the_k] + cost <= b[knap] and duration[the_k] + time <= t[knap] and sum_solution[the_k] + weight >= sum_solution[the_k]:
                inventory += -1
                budget[the_k] += cost
                duration[the_k] += time
                solution[the_k].append(item_no)
                sum_solution[the_k] += weight
            k_loop += 1
    for k in range(n):
        the_k = 'k_' + str(k)
        z += sum_solution[the_k]
    return z

In [5]:
def get_max_index(df):
    col = df.max(axis = 0, skipna=True).sort_values(ascending = False).index[0]
    index = df[col].sort_values(ascending = False).index[0]
    return (index, col)

In [6]:
def alg3():
    z = 0
    index_3 = ['i_' + str(i) for i in range(m)]
    columns_3 = ['k_' + str(k) for k in range(n)]
    matrix_3 = np.array(w)
    table_3 = pd.DataFrame(matrix_3, index = index_3, columns = columns_3)
    budget = {}
    duration = {}
    solution ={}
    sum_solution={}
    for k in range(n):
        knap_no = k
        budget["k_{0}".format(k)] = 0
        duration["k_{0}".format(k)] = 0
        solution["k_{0}".format(k)] = []
        sum_solution["k_{0}".format(k)] = 0
    interations = m * n
    library = np.array(e)
    while interations > 0:
        tpl = get_max_index(table_3)
        item = int(tpl[0][2:])
        knap = int(tpl[1][2:])
        the_k = tpl[1]
        inventory = library[item]
        cost = c[item]
        time = d[item]
        weight = table_3.iloc[item, knap]
        if inventory > 0 and budget[the_k] + cost <= b[knap] and duration[the_k] + time <= t[knap] and sum_solution[the_k] + weight >= sum_solution[the_k]:
            library[item] += -1
            budget[the_k] += cost
            duration[the_k] += time
            solution[the_k].append(item)
            sum_solution[the_k] += weight
            table_3.iloc[item, knap] = None
        else: 
            table_3.iloc[item, knap] = None
        interations += -1
    for k in range(n):
        the_k = 'k_' + str(k)
        z += sum_solution[the_k]
    return z

In [7]:
def get_min_index(df):
    col = df.min(axis = 0, skipna=True).sort_values(ascending = True).index[0]
    index = df[col].sort_values(ascending = True).index[0]
    return (index, col)

In [8]:
def alg4():
    z = 0
    index_4 = ['i_' + str(i) for i in range(m)]
    columns_4 = ['k_' + str(k) for k in range(n)]
    matrix_4 = np.array(w)
    table_4 = pd.DataFrame(matrix_4, index = index_4, columns = columns_4)
    budget = {}
    duration = {}
    solution ={}
    sum_solution={}
    for k in range(n):
        knap_no = k
        budget["k_{0}".format(k)] = 0
        duration["k_{0}".format(k)] = 0
        solution["k_{0}".format(k)] = []
        sum_solution["k_{0}".format(k)] = 0
    interations = m * n
    library = np.array(e)
    iteration_list = table_4
    iteration_result_list = table_4
    while interations > 0:
        tpl = get_min_index(iteration_list)
        item = int(tpl[0][2:])
        old_knap = int(tpl[1][2:])
        inventory = library[item]
        cost = c[item]
        time = d[item]
        the_k = table_4.idxmax(1)[item]
        knap = int(the_k[2:])
        weight = table_4.iloc[item, knap]
        if inventory > 0 and budget[the_k] + cost <= b[knap] and duration[the_k] + time <= t[knap] and sum_solution[the_k] + weight >= sum_solution[the_k]:
            library[item] += -1
            budget[the_k] += cost
            duration[the_k] += time
            solution[the_k].append(item)
            sum_solution[the_k] += weight
            iteration_list.iloc[item, old_knap] = None
        else: 
            iteration_list.iloc[item, old_knap] = None
        interations += -1
    for k in range(n):
        the_k = 'k_' + str(k)
        z += sum_solution[the_k]
    return z

In [9]:
def algOptimal():
    simplex = pulp.LpProblem('Simplex', pulp.LpMaximize) 
    x_var = np.array([[pulp.LpVariable("x_{}_{}".format(i, k), cat='Binary') for k in range(n)] for i in range(m)])
    w_array = np.array(w)
    simplex += sum(x_var[i, k] * w_array[i, k] for k in range(n) for i in range(m))
    for k in range (n): #restrições da mochila
        simplex += (sum(x_var[i, k] * d[i] for i in range(m)) <= t[k])
        simplex += (sum(x_var[i, k] * c[i] for i in range(m)) <= b[k])
    for i in range (m) : #restrições de itens
        simplex += (sum(x_var[i, k] for k in range(n)) <= e[i])
    simplex.solve()
    return pulp.value(simplex.objective)

In [10]:
alg1_start = time.time()
response = alg1()
alg1_finish = time.time()
print('alg1', response, alg1_finish - alg1_start)
alg2_start = time.time()
response = alg2()
alg2_finish = time.time()
print('alg2', response, alg2_finish - alg2_start)
alg3_start = time.time()
response = alg3()
alg3_finish = time.time()
print('alg3', response, alg3_finish - alg3_start)
alg4_start = time.time()
response = alg4()
alg4_finish = time.time()
print('alg4', response, alg4_finish - alg4_start)
#algOptimal_start = time.time()
#response = algOptimal()
algOptimal_finish = time.time()
#print('algOptimal', response, algOptimal_finish - algOptimal_start)
print('total', algOptimal_finish - alg1_start)

('alg1', 12740, 3.1354029178619385)
('alg2', 13861, 1.5498578548431396)
('alg3', 22946.0, 267.7750859260559)


KeyboardInterrupt: 